In [94]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from tiny_ta import atr, rsi, sma

# Reversal - Strategie im S&P 500

Zunächst werden die Kursdaten für den S&P 500 geladen und bereinigt

In [95]:
SYMBOL = "^GSPC"
stock = yf.download(SYMBOL)

stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

[*********************100%%**********************]  1 of 1 completed


Ein paar Indikatoren für spätere Auswertungen

In [96]:
stock["rsi_2"] = rsi(stock.Close, 2)
stock["rsi_3"] = rsi(stock.Close, 3)
stock["rsi_7"] = rsi(stock.Close, 7)
stock["rsi_10"] = rsi(stock.Close, 10)

stock["atr_10"] = atr(stock, 10, smoothing="rma") / stock.Close * 100

adx_intervall = 21
adx = ta.adx(stock.High, stock.Low, stock.Close, adx_intervall)
stock["adx"] = adx[f"ADX_{adx_intervall}"]
stock["adx_direction"] = np.where(
    adx[f"DMP_{adx_intervall}"] > adx[f"DMN_{adx_intervall}"], 1, -1
)

stock["adx_10"] = ta.adx(stock.High, stock.Low, stock.Close, 10, 10)["ADX_10"]


stock["sma"] = sma(stock.Close, 20)
stock["sma_20"] = stock.Close / sma(stock.Close, 20)
stock["sma_50"] = stock.Close / sma(stock.Close, 50)
stock["sma_100"] = stock.Close / sma(stock.Close, 100)
stock["sma_200"] = stock.Close / sma(stock.Close, 200)

stock["sma_pct"] = (stock.Close / stock.sma).round(2)

stock["month"] = stock.index.month

df = stock

## Jetzt zur Strategie
Es wird gekauft wenn der RSI unter 30 gesprungen ist und bei einem RSI über 50 wieder verkauft

In [97]:
from backtesting import Strategy


class reversal(Strategy):
    """
    strategy for reversals
    """

    rsi_exit = 50
    rsi_before = 30
    rsi_entry = 15

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )

    def init(self):
        super().init()

        self.rsi = self.I(lambda: self.data.rsi_2, name="RSI(2)", overlay=False)

    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for an existing trade
        for trade in self.trades:
            if trade.is_long:
                if self.rsi[-1] > self.rsi_exit:
                    trade.close()

        if self.rsi[-1] < self.rsi_entry and self.rsi[-2] < self.rsi_before:
            self.buy()

In [98]:
from backtesting import Backtest

bt = Backtest(df["2012-01-01":], reversal, cash=100_000, commission=0.002)
stats = bt.run()
bt.plot(superimpose=False)  # , open_browser=False)
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                   18.128844
Equity Final [$]                 116733.94435
Equity Peak [$]                 124956.617118
Return [%]                          16.733944
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    1.270261
Volatility (Ann.) [%]                 7.89576
Sharpe Ratio                         0.160879
Sortino Ratio                        0.227445
Calmar Ratio                         0.077296
Max. Drawdown [%]                  -16.433815
Avg. Drawdown [%]                   -2.191753
Max. Drawdown Duration     2247 days 00:00:00
Avg. Drawdown Duration      156 days 00:00:00
# Trades                                  153
Win Rate [%]                        64.705882
Best Trade [%]                       4.624744
Worst Trade [%]                      -4.77656
Avg. Trade [%]                    

jetzt wird überprüft welche Tage für die Ein- und Ausstiege besser sein könnten.

In [99]:
stats, heatmap = bt.optimize(
    rsi_entry=range(4, 15),
    rsi_exit=range(20, 50, 1),
    rsi_before=range(5, 30, 1),
    maximize="Profit Factor",  # "Equity Final [$]",
    max_tries=200,
    random_state=0,
    return_heatmap=True,
)

/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [100]:
# heatmap
heatmap.sort_values().iloc[-3:]

rsi_entry  rsi_exit  rsi_before
7          43        15            1.693653
6          44        9             1.797058
           43        14            1.808477
Name: Profit Factor, dtype: float64

In [101]:
heatmap.groupby(["rsi_entry", "rsi_exit", "rsi_before"]).mean().unstack()

rsi_before                5   6   7         8         9         10  11  12  \
rsi_entry rsi_exit                                                           
4         20             NaN NaN NaN  0.969531       NaN       NaN NaN NaN   
          23             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
          24             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
          26        0.763406 NaN NaN       NaN       NaN       NaN NaN NaN   
          27             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
...                      ...  ..  ..       ...       ...       ...  ..  ..   
14        38             NaN NaN NaN       NaN  1.560269  1.458885 NaN NaN   
          40             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
          45             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
          46             NaN NaN NaN       NaN       NaN       NaN NaN NaN   
          49        1.324264 NaN NaN       NaN       NaN       NaN NaN NaN   

rsi_before          13  14  ...  20       21        22  23  24        25  \
rsi_entry rsi_exit          ...                                            
4         20       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
          23       NaN NaN  ... NaN      NaN  0.903234 NaN NaN       NaN   
          24       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
          26       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
          27       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
...                 ..  ..  ...  ..      ...       ...  ..  ..       ...   
14        38       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
          40       NaN NaN  ... NaN      NaN       NaN NaN NaN  1.095016   
          45       NaN NaN  ... NaN  1.13732       NaN NaN NaN       NaN   
          46       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   
          49       NaN NaN  ... NaN      NaN       NaN NaN NaN       NaN   

rsi_before                26  27  28  29  
rsi_entry rsi_exit                        
4         20             NaN NaN NaN NaN  
          23             NaN NaN NaN NaN  
          24             NaN NaN NaN NaN  
          26             NaN NaN NaN NaN  
          27        0.941535 NaN NaN NaN  
...                      ...  ..  ..  ..  
14        38             NaN NaN NaN NaN  
          40             NaN NaN NaN NaN  
          45             NaN NaN NaN NaN  
          46             NaN NaN NaN NaN  
          49             NaN NaN NaN NaN  

[154 rows x 25 columns]

In [102]:
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                    7.640013
Equity Final [$]                122978.721056
Equity Peak [$]                 123518.967638
Return [%]                          22.978721
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    1.701723
Volatility (Ann.) [%]                5.449956
Sharpe Ratio                         0.312245
Sortino Ratio                        0.479781
Calmar Ratio                          0.13538
Max. Drawdown [%]                  -12.569997
Avg. Drawdown [%]                   -1.898414
Max. Drawdown Duration     1303 days 00:00:00
Avg. Drawdown Duration      132 days 00:00:00
# Trades                                   66
Win Rate [%]                        71.212121
Best Trade [%]                       4.620926
Worst Trade [%]                      -4.77656
Avg. Trade [%]                    

The Winner is ...

In [103]:
stats["_strategy"]

<Strategy reversal(rsi_entry=6,rsi_exit=43,rsi_before=14)>

In [104]:
stats = bt.run(rsi_entry=6, rsi_exit=43, rsi_before=14)
bt.plot(superimpose=False, open_browser=False)

Row(id='16968', ...)

In [105]:
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                    7.640013
Equity Final [$]                122978.721056
Equity Peak [$]                 123518.967638
Return [%]                          22.978721
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    1.701723
Volatility (Ann.) [%]                5.449956
Sharpe Ratio                         0.312245
Sortino Ratio                        0.479781
Calmar Ratio                          0.13538
Max. Drawdown [%]                  -12.569997
Avg. Drawdown [%]                   -1.898414
Max. Drawdown Duration     1303 days 00:00:00
Avg. Drawdown Duration      132 days 00:00:00
# Trades                                   66
Win Rate [%]                        71.212121
Best Trade [%]                       4.620926
Worst Trade [%]                      -4.77656
Avg. Trade [%]                    

Die Win Rate beim Trading bezieht sich auf das Verhältnis der Anzahl der gewonnenen Trades zur Gesamtanzahl der Trades. 
Wenn jemand beispielsweise 70% seiner Trades gewinnt, beträgt seine Win Rate 70%.

In [106]:
stats["Win Rate [%]"]

71.21212121212122

Durchschnittlicher Gewinn in Prozent. 
Wie viel Gewinn oder Verlust wurden im Durchschnitt pro Trade erzielt.

In [107]:
stats["Avg. Trade [%]"]

0.32012640138896487

Der Profit-Faktor stellt die Verhältnisse zwischen Gewinnen und Verlusten dar.
Der Profit-Faktor wird berechnet, indem die Summe aller Profite (Gewinne) durch die Summe aller Verluste (einschließlich Gebühren und Kommissionen) für den gesamten Handelszeitraum geteilt wird.
Ein Profit-Faktor größer als 1 zeigt an, dass die Strategie profitabel ist. Je höher der Wert, desto besser.

Ein guter Profit-Faktor liegt normalerweise zwischen 1,75 und 41.
Ein Wert über 1,75 zeigt eine solide Strategie an, während Werte über 4 außergewöhnlich gut sind.

In [108]:
stats["Profit Factor"]

1.8084767811047255

In [109]:
trades = stats["_trades"]

for index, row in trades.iterrows():
    start_range = stock[: row["EntryTime"]]
    trades.loc[index, "SignalTime"] = start_range.index.values[-2]

merged = pd.merge(
    trades,
    df[
        [
            "adx_10",
            "adx_direction",
            "rsi_3",
            "rsi_7",
            "rsi_10",
            "sma_20",
            "sma_50",
            "sma_100",
            "sma_200",
        ]
    ],
    left_on="SignalTime",
    right_index=True,
)
# merged["win"] = np.where(merged.PnL > 0, 1, -1)
# merged["win"] = np.where(merged.PnL > 0, 1, -1)

In [110]:
trades = stats["_trades"]

for index, row in trades.iterrows():
    start_range = stock[: row["EntryTime"]]
    trades.loc[index, "SignalTime"] = start_range.index.values[-2]

merged = pd.merge(
    trades,
    df[
        [
            "adx_10",
            "adx_direction",
            "rsi_3",
            "rsi_7",
            "rsi_10",
            "sma_20",
            "sma_50",
            "sma_100",
            "sma_200",
        ]
    ],
    left_on="SignalTime",
    right_index=True,
)

In [111]:
pd.DataFrame(merged.groupby(merged.ExitTime.dt.year).PnL.sum().round(1))

,PnL
ExitTime,
2012,1273.6
2013,2405.3
2014,2770.7
2015,4097.5
2016,801.8
2017,246.5
2018,-5173.4
2019,2611.9
2020,-1773.2


In [112]:
equity = stats["_equity_curve"]
equity.groupby(equity.index.year).Equity.last().round(1)

2012    102782.3
2013    103678.8
2014    106449.6
2015    110547.0
2016    111348.8
2017    111595.3
2018    106421.9
2019    109033.8
2020    107260.6
2021    111751.4
2022    118725.8
2023    122100.8
2024    122978.7
Name: Equity, dtype: float64